In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import ephem
from sbpy.data import Ephem
import astropy.units as u
from astropy.time import Time

In [4]:
apo = ephem.Observer()
apo.lat = '32:46:49'
apo.lon = '-105:49:13'
apo.elevation = 2788
# UTC night of interest (APO = -7 hrs to UTC most of the time .. midnight UTC is around or before sunset)
apo.date = '2020/01/12 13:00:00'
#civil twilight -6, nautical twilight -12, astronomical twilight -18
apo.pressure = 0   # set pressure to 0 and horizon to -0:34 from whatever, for refraction magic (well .. approximate?)
apo.horizon = '-12:34'

sunset = apo.next_setting(ephem.Sun(), use_center=True)
sunrise = apo.next_rising(ephem.Sun(), use_center=True)

In [6]:
# UTC sunset sunrise .. but this seems like it might not be quite right? Close.
print("UTC Sunset ", sunset)
print("Sunset converted to your current local time ", ephem.localtime(sunset))
print("UTC Sunrise ", sunrise)

UTC Sunset  2020/1/13 01:16:58
Sunset converted to your current local time  2020-01-12 17:16:57.000006
UTC Sunrise  2020/1/12 13:06:15


In [7]:
# NICFPS: 1024x1024 HgCdTe device with a 0.273 arsec/pixel scale -- 4.58 arcmin across FOV

In [9]:
# Get target information from Horizons.

targets = ['2020 AV2']

start = Time('2020-01-13T0:00:00')
end = Time('2020-01-13T03:00:00')

ephs = Ephem.from_horizons(targets, location=705, 
                          epochs={'start': start,
                                  'stop': end,
                                  'step':5*u.min})

In [10]:
ephs.table[0:20]

targetname,H,G,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA*cos(Dec)_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat,epoch
,mag,,,,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,h,,mag,mag,%,arcsec,arcsec,,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,,deg,deg,%,deg,deg,deg,deg,,s,deg,deg,deg,deg,deg,deg,,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,,deg,deg,deg,
str10,float64,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object
(2020 AV2),16.26,0.15,*,,326.05327,-1.38036,326.30269,-1.28989,60.76042,71.55183,235.8273,39.3001,648.15,-626.19,108063.7,73125.73,61.475,0.4082757405,1.576,0.216,17.87,44.957,0.0,134744.9,*,0.0,0.0,0.0,0.0,0.0,234.8,0.0,0.0,0.0,66.8294,13.8694,0.600758999756,-5.8251524,0.72029430201894,-15.4280538,5.990505,36.81644,20.58163,37.4291,/T,95.7929,164.2,93.3,46.7714,54.856,109.894,-21.09242,Aqr,69.184249,328.0935876,11.5341663,0.0,0.0,54.664229,-38.311128,16.8071243089,0.000354,6.194,5.9,6.197,5.896,6.387,229.6,8.554,1075916.24,0.1321522,2094.9,7613.8,7.177741,233.7691,2.654763392,95.7886,17.0149,19.1387,2458861.5
(2020 AV2),16.26,0.15,*,,326.05468,-1.37871,326.30409,-1.28823,60.78258,71.54396,236.9761,38.4244,639.91,-634.61,108056.3,73129.57,61.471,0.4918372353,1.606,0.22,17.87,44.953,0.0,134740.5,*,0.0,0.0,0.0,0.0,0.0,234.8,0.0,0.0,0.0,66.8365,13.8703,0.600747317014,-5.8256704,0.72026336566812,-15.4219717,5.990248,36.81722,20.57749,37.4279,/T,95.7973,164.2,93.3,46.7683,54.853,109.884,-21.09302,Aqr,69.184249,328.095509,11.5352505,0.0,0.0,54.666982,-38.311361,16.8904366308,0.000354,6.202,5.907,6.205,5.903,6.274,230.14,8.564,1075950.42,0.1321071,2094.18,7611.19,7.177969,233.776,2.738231141,95.793,17.0193,19.1411,2458861.503472222
(2020 AV2),16.26,0.15,*,,326.05609,-1.37706,326.3055,-1.28658,60.8057,71.53606,238.0966,37.5375,631.91,-642.57,108048.9,73133.41,61.468,0.5753987301,1.638,0.224,17.87,44.949,0.0,134736.1,*,0.0,0.0,0.0,0.0,0.0,234.79,0.0,0.0,0.0,66.8436,13.8713,0.600735633234,-5.8261882,0.72023244139327,-15.4160121,5.989991,36.818,20.57349,37.4267,/T,95.8017,164.2,93.2,46.7652,54.851,109.873,-21.09363,Aqr,69.184249,328.0974308,11.5363344,0.0,0.0,54.669734,-38.311595,16.9737488871,0.000354,6.21,5.913,6.213,5.91,6.163,230.69,8.575,1075984.59,0.1320621,2093.46,7608.58,7.178196,233.783,2.821698857,95.7974,17.0238,19.1435,2458861.506944444
(2020 AV2),16.26,0.15,*,,326.0575,-1.3754,326.30691,-1.28492,60.82978,71.52813,239.1901,36.6397,624.15,-650.11,108041.6,73137.25,61.464,0.6589602249,1.672,0.229,17.87,44.946,0.0,134731.6,*,0.0,0.0,0.0,0.0,0.0,234.79,0.0,0.0,0.0,66.8507,13.8722,0.600723948415,-5.826706,0.72020152894565,-15.410178,5.989733,36.81878,20.56966,37.4255,/T,95.8061,164.3,93.2,46.762,54.848,109.863,-21.09423,Aq

In [11]:
# Narrow down ephemerides to times when the target is desireable. 
# Let's cut off at elevation < 10 and Civil twilight, so there's a good buffer.
ephs = ephs[np.where((ephs['elevation'] > 10*u.deg) & (ephs['solar_presence'] != '*'))]
times = Time(ephs['epoch'], format='mjd', scale='utc')

In [12]:
# How far does it move over that period?  
# NICFPS: 1024x1024 HgCdTe device with a 0.273 arsec/pixel scale -- 4.58 arcmin across FOV
print((ephs['RA'] - ephs['RA'][0]).to(u.arcmin))
print((ephs['DEC'] - ephs['DEC'][0]).to(u.arcmin))

# So .. maybe just pick a pointing and stick with that for entire period? 

[0.     0.0846 0.1698 0.2544 0.339  0.4242 0.5094 0.594  0.6792 0.7644
 0.8496 0.9348 1.02   1.1052 1.1904 1.2756 1.3614 1.4466 1.5324 1.6176
 1.7034 1.7892 1.875  1.9608] arcmin
[0.     0.0996 0.1986 0.2976 0.3972 0.4962 0.5952 0.6942 0.7938 0.8928
 0.9918 1.0908 1.1898 1.2888 1.3884 1.4874 1.5864 1.6854 1.7844 1.8834
 1.9824 2.0814 2.1804 2.2794] arcmin


In [13]:
print("Object set time (UTC) ", times.max().isot)

Object set time (UTC)  2020-01-13T02:25:00.000


In [14]:
cols = ['RA', 'DEC', 'RA_3sigma', 'DEC_3sigma', 'V', 'airmass', 'epoch' , 'EL', 'solar_presence', 'lunar_elong']
ephs[cols]

RA,DEC,RA_3sigma,DEC_3sigma,V,airmass,epoch,EL,solar_presence,lunar_elong
deg,deg,arcsec,arcsec,mag,,,deg,,deg
float64,float64,float64,float64,float64,float64,object,float64,str1,float64
326.06173,-1.37044,6.241,5.939,17.87,1.788,2458861.520833333,33.8876,C,164.3
326.06314,-1.36878,6.248,5.946,17.87,1.833,2458861.524305556,32.9523,C,164.3
326.06456,-1.36713,6.256,5.952,17.87,1.881,2458861.527777778,32.0088,C,164.3
326.06597,-1.36548,6.264,5.959,17.87,1.932,2458861.53125,31.0577,N,164.3
326.06738,-1.36382,6.271,5.965,17.87,1.987,2458861.534722222,30.0993,N,164.3
326.0688,-1.36217,6.279,5.972,17.87,2.046,2458861.538194444,29.134,N,164.3
326.07022,-1.36052,6.287,5.978,17.87,2.11,2458861.541666667,28.1622,N,164.3
326.07163,-1.35887,6.294,5.985,17.87,2.179,2458861.545138889,27.1843,N,164.3


In [15]:
# TUI commands to slew to target.
for i in range(len(ephs)):
    print(f"{ephs[i]['epoch'].isot} tcc track {ephs[i]['RA'].value}, {ephs[i]['DEC'].value}, \
{ephs[i]['RA*cos(Dec)_rate'].to(u.deg/u.second).value}, {ephs[i]['DEC_rate'].to(u.deg/u.second).value} \
Fk5=2000.0 /Rotangle=0.0 /Rottype=Object")

2020-01-13T00:30:00.000 tcc track 326.06173, -1.37044, 4.699657407407407e-06, 5.517298611111111e-06 Fk5=2000.0 /Rotangle=0.0 /Rottype=Object
2020-01-13T00:35:00.000 tcc track 326.06314, -1.36878, 4.701795524691358e-06, 5.516679012345679e-06 Fk5=2000.0 /Rotangle=0.0 /Rottype=Object
2020-01-13T00:40:00.000 tcc track 326.06456, -1.36713, 4.703999228395062e-06, 5.516057098765433e-06 Fk5=2000.0 /Rotangle=0.0 /Rottype=Object
2020-01-13T00:45:00.000 tcc track 326.06597, -1.36548, 4.706266975308642e-06, 5.51543287037037e-06 Fk5=2000.0 /Rotangle=0.0 /Rottype=Object
2020-01-13T00:50:00.000 tcc track 326.06738, -1.36382, 4.708597993827161e-06, 5.514807098765433e-06 Fk5=2000.0 /Rotangle=0.0 /Rottype=Object
2020-01-13T00:55:00.000 tcc track 326.0688, -1.36217, 4.710989197530864e-06, 5.514178240740741e-06 Fk5=2000.0 /Rotangle=0.0 /Rottype=Object
2020-01-13T01:00:00.000 tcc track 326.07022, -1.36052, 4.713440586419753e-06, 5.513547839506173e-06 Fk5=2000.0 /Rotangle=0.0 /Rottype=Object
2020-01-13T01:0

In [ ]:
# exposures - on-sky + readout (NFS=8 JHK, NFS=0 Y)
exptime = {}
exptime['Y'] = 100 + 5.4
exptime['J'] = 50 + 10.9
exptime['H'] = 20 + 10.9
exptime['K'] = 10 + 10.9



In [52]:
# Middle of range of  likely asteroid colors
# Y-J ~0.4, Y-H ~ 0.5, Y-K ~0.7 (redder)
dmag = 0.5
fluxratio = 10**(dmag / 2.5)
print(f"Change in magnitude of {dmag} means that the flux changed by a factor of {fluxratio}")
# For same SNR need (fluxratio)**2 exposure time (assuming photon limited)
exptimeratio = 1/fluxratio**2
print(f"To maintain SNR (without sensitivity change), change exposure {exptimeratio} longer")

Change in magnitude of 0.5 means that the flux changed by a factor of 1.5848931924611136
To maintain SNR (without sensitivity change), change exposure 0.3981071705534972 longer


In [57]:
# NIC-FPS m5 values for 1 hour exposure time
apo_m5= {}
apo_m5['Z'] = 24.2
apo_m5['J'] = 23.3
apo_m5['H'] = 22.5
apo_m5['K'] = 21.9
# Middle of range of  likely asteroid colors
# Y-J ~0.4, Y-H ~ 0.5, Y-K ~0.7 (redder)
asteroid = {}
asteroid['Z'] = 0
asteroid['J'] = -0.4
asteroid['H'] = -0.5
asteroid['K'] = -0.7

In [84]:
for f1, f2 in zip(['Z', 'Z', 'Z'], ['J', 'H', 'K']):
    print('dmag apo m5 (sensitivity)', f1,'-', f2, apo_m5[f1] - apo_m5[f2])
    print('dmag asteroid (color)', f1,'-', f2, asteroid[f1] - asteroid[f2])
    print('Diff (& exptime ratio)', f1, '-', f2, (asteroid[f1] - asteroid[f2]) - (apo_m5[f1] - apo_m5[f2]),
         (10**(-(asteroid[f1] - asteroid[f2] - (apo_m5[f1] - apo_m5[f2]))/2.5))**2)

dmag apo m5 (sensitivity) Z - J 0.8999999999999986
dmag asteroid (color) Z - J 0.4
Diff (& exptime ratio) Z - J -0.49999999999999856 2.5118864315095735
dmag apo m5 (sensitivity) Z - H 1.6999999999999993
dmag asteroid (color) Z - H 0.5
Diff (& exptime ratio) Z - H -1.1999999999999993 9.120108393559086
dmag apo m5 (sensitivity) Z - K 2.3000000000000007
dmag asteroid (color) Z - K 0.7
Diff (& exptime ratio) Z - K -1.6000000000000008 19.054607179632498
